In [1]:
import pandas as pd

data = pd.ExcelFile("ARTIvir_CoV_minimal_combined_dset.xlsx")

In [2]:
data.sheet_names;

In [3]:
dfs =  {sheet_name: data.parse(sheet_name) for sheet_name in data.sheet_names}

In [4]:
virus_host_PPI = dfs["PPIs"]
effectome = dfs["EFF"]
proteome = dfs["NHBE_FPs"]
phosphoproteome = dfs["PPx"]
ubiquitinome = dfs["Ux"]
transcriptome = dfs["Tx"]

# Proteome complete data analysis

In [11]:
proteome.head()
#len(proteome["Gene.names"])

,Majority.protein.IDs,Gene.names,lambda.min.converged,Lasso_SARSCoV.min,Lasso_SARSCoV.min.p,Lasso_SARSCoV2.min,Lasso_SARSCoV2.min.p,lambda.min,external_synonym,Lasso_SARSCoV.min.sig,Lasso_SARSCoV2.min.sig,rn_name
0,sp|A0A0U1RRL7|MMPOS_HUMAN,MMP24OS,1.0,0.000000,NaN,0.000000,NaN,0.015477,MMP24OS;MMP24-AS1,not_sig,not_sig,NaN
1,sp|A0AV96-2|RBM47_HUMAN;sp|A0AV96|RBM47_HUMAN,RBM47,1.0,0.614245,0.003332,0.913143,6.281934e-15,0.002456,RBM47;FLJ20273;NET18,low_sig,high_sig,NaN
2,sp|A0AVT1|UBA6_HUMAN;sp|A0AVT1-2|UBA6_HUMAN,UBA6,1.0,-0.042801,0.474667,-0.077215,1.452895e-01,0.000078,UBA6;FLJ10808;UBE1L2,not_sig,not_sig,UBA6
3,sp|A0FGR8-2|ESYT2_HUMAN;sp|A0FGR8|ESYT2_HUMAN;...,ESYT2,1.0,0.000000,NaN,0.000000,NaN,0.015747,ESYT2;CHR2SYT;FAM62B;KIAA1228,not_sig,not_sig,NaN
4,sp|A0JNW5|UH1BL_HUMAN,UHRF1BP1L,1.0,0.000000,NaN,0.000000,NaN,0.026649,UHRF1BP1L;KIAA0701,not_sig,not_sig,NaN


In [12]:
sars_cov_proteome_nan = proteome["Lasso_SARSCoV.min.p"].isnull()
print(f"complete data for proteome in SARS-CoV {len(sars_cov_proteome_nan) - sars_cov_proteome_nan.sum()} on a total of {len(sars_cov_proteome_nan)}")

complete data for proteome in SARS-CoV 3166 on a total of 6129


In [13]:
sars_cov_2_proteome_nan = proteome["Lasso_SARSCoV2.min.p"].isnull()
print(f"complete data for proteome in SARS-CoV {len(sars_cov_2_proteome_nan) - sars_cov_2_proteome_nan.sum()} on a total of {len(sars_cov_2_proteome_nan)}")

complete data for proteome in SARS-CoV 3812 on a total of 6129


In [14]:
sars_cov_complete_genes = proteome["Gene.names"][sars_cov_proteome_nan == False]
len(sars_cov_complete_genes)

3166

In [15]:
sars_cov_2_complete_genes = proteome["Gene.names"][sars_cov_2_proteome_nan == False]

In [16]:
all_proteins_list = proteome["Gene.names"]

# Transcriptome complete data analysis

In [12]:
transcriptome.head()
len(transcriptome["gene_name"])

13108

In [17]:
num_nan_values_sars_cov_2 = transcriptome["fold_change_log2.SARS_CoV2@6h_vs_mock@6h"].isnull().sum() + transcriptome["fold_change_log2.SARS_CoV2@12h_vs_mock@12h"].isnull().sum() + transcriptome["fold_change_log2.SARS_CoV2@24h_vs_mock@24h"].isnull().sum()
num_nan_values_sars_cov = transcriptome["fold_change_log2.SARS_CoV@6h_vs_mock@6h"].isnull().sum() + transcriptome["fold_change_log2.SARS_CoV@12h_vs_mock@12h"].isnull().sum() + transcriptome["fold_change_log2.SARS_CoV@24h_vs_mock@24h"].isnull().sum()

print(f"number of nan values for SARS-CoV-2 {num_nan_values_sars_cov_2}")
print(f"number of nan values for SARS-CoV {num_nan_values_sars_cov}")

number of nan values for SARS-CoV-2 0
number of nan values for SARS-CoV 0


# P-value definition:
## the probability that a random sample from the posterior distribution of the first condition would be smaller (or larger) than a random sample drawn from the second condition. 

Case of SARS-CoV-2:

In [18]:
def omic_completeness_analyzer(omic, which_sars: str):
    gene_list_6h = omic["gene_name"][omic["p_value." + which_sars + "@6h_vs_mock@6h"] < 0.05].values
    gene_list_12h = omic["gene_name"][omic["p_value." + which_sars + "@12h_vs_mock@12h"] < 0.05].values
    gene_list_24h = omic["gene_name"][omic["p_value." + which_sars + "@24h_vs_mock@24h"] < 0.05].values

    import numpy as np
    shared_genes_with_good_p_value = []
    for item in gene_list_6h:
        if (item in gene_list_12h) & (item in gene_list_24h):
            shared_genes_with_good_p_value.append(item)
    shared_genes_with_good_p_value = np.array(shared_genes_with_good_p_value)
    
    unique_genes_with_good_p_value = []
    for item in gene_list_6h:
        if item not in unique_genes_with_good_p_value:
            unique_genes_with_good_p_value.append(item)
    for item in gene_list_12h:
        if item not in unique_genes_with_good_p_value:
            unique_genes_with_good_p_value.append(item) 
    for item in gene_list_24h:
        if item not in unique_genes_with_good_p_value:
            unique_genes_with_good_p_value.append(item)        
    return gene_list_6h, gene_list_12h, gene_list_24h, shared_genes_with_good_p_value, unique_genes_with_good_p_value

gene_list_6h_cov2, gene_list_12h_cov2, gene_list_24h_cov2, shared_genes_with_good_p_value_cov2, unique_genes_with_good_p_value_cov2  = omic_completeness_analyzer(transcriptome, "SARS_CoV2")
print(f"In SARS-CoV-2 we have regulated genes with p-value < 0.05 shared among the time points 6h,12h,24h = {len(shared_genes_with_good_p_value_cov2)} on a total of {len(gene_list_6h_cov2)} at 6h of {len(gene_list_12h_cov2)} at 12h of {len(gene_list_24h_cov2)} at 24h")
#transcriptome[transcriptome["gene_name"] == "AL022718.1"][["p_value.SARS_CoV2@6h_vs_mock@6h", "fold_change_log2.SARS_CoV2@6h_vs_mock@6h", "p_value.SARS_CoV2@12h_vs_mock@12h", "fold_change_log2.SARS_CoV2@12h_vs_mock@12h", "p_value.SARS_CoV2@24h_vs_mock@24h", "fold_change_log2.SARS_CoV2@24h_vs_mock@24h"]]
print(f"Unique genes with good p-values {len(unique_genes_with_good_p_value_cov2)} on a total of {len(gene_list_6h_cov2) + len(gene_list_12h_cov2) + len(gene_list_24h_cov2)}")

In SARS-CoV-2 we have regulated genes with p-value < 0.05 shared among the time points 6h,12h,24h = 200 on a total of 1249 at 6h of 1778 at 12h of 1399 at 24h
Unique genes with good p-values 3475 on a total of 4426


Case of SARS-CoV:

In [19]:
gene_list_6h_cov, gene_list_12h_cov, gene_list_24h_cov, shared_genes_with_good_p_value_cov, unique_genes_with_good_p_value_cov = omic_completeness_analyzer(transcriptome, "SARS_CoV")
print(f"In SARS-CoV we have regulated genes with p-value < 0.05 shared among the time points 6h,12h,24h = {len(shared_genes_with_good_p_value_cov)} on a total of {len(gene_list_6h_cov)} at 6h of {len(gene_list_12h_cov)} at 12h of {len(gene_list_24h_cov)} at 24h")
print(f"Unique genes with good p-values {len(unique_genes_with_good_p_value_cov)} on a total of {len(gene_list_6h_cov) + len(gene_list_12h_cov) + len(gene_list_24h_cov)}")

#transcriptome[transcriptome["gene_name"] == "AL022718.1"][["p_value.SARS_CoV2@6h_vs_mock@6h", "fold_change_log2.SARS_CoV2@6h_vs_mock@6h", "p_value.SARS_CoV2@12h_vs_mock@12h", "fold_change_log2.SARS_CoV2@12h_vs_mock@12h", "p_value.SARS_CoV2@24h_vs_mock@24h", "fold_change_log2.SARS_CoV2@24h_vs_mock@24h"]]


In SARS-CoV we have regulated genes with p-value < 0.05 shared among the time points 6h,12h,24h = 212 on a total of 1059 at 6h of 1597 at 12h of 2327 at 24h
Unique genes with good p-values 3876 on a total of 4983


# Ubiquitinome complete data analysis

In [20]:
ubiquitinome.columns;

In [21]:
num_nan_values_sars_cov_2 = ubiquitinome["fold_change_log2.SARS_CoV2@6h_vs_mock@6h"].isnull().sum() + ubiquitinome["fold_change_log2.SARS_CoV2@12h_vs_mock@12h"].isnull().sum() + ubiquitinome["fold_change_log2.SARS_CoV2@24h_vs_mock@24h"].isnull().sum()
num_nan_values_sars_cov = ubiquitinome["fold_change_log2.SARS_CoV@6h_vs_mock@6h"].isnull().sum() + ubiquitinome["fold_change_log2.SARS_CoV@12h_vs_mock@12h"].isnull().sum() + ubiquitinome["fold_change_log2.SARS_CoV@24h_vs_mock@24h"].isnull().sum()

print(f"number of nan values for SARS-CoV-2 {num_nan_values_sars_cov_2}")
print(f"number of nan values for SARS-CoV {num_nan_values_sars_cov}")

number of nan values for SARS-CoV-2 0
number of nan values for SARS-CoV 0


Case of SARS-CoV-2:

In [22]:
gene_list_6h_cov2, gene_list_12h_cov2, gene_list_24h_cov2, ubi_shared_genes_with_good_p_value_cov2, ubi_unique_genes_with_good_p_value_cov2 = omic_completeness_analyzer(ubiquitinome, "SARS_CoV2")
print(f"In SARS-CoV-2 we have ubiquitinome regulated genes with p-value < 0.05 shared among the time points 6h,12h,24h = {len(ubi_shared_genes_with_good_p_value_cov2)} on a total of {len(gene_list_6h_cov2)} at 6h of {len(gene_list_12h_cov2)} at 12h of {len(gene_list_24h_cov2)} at 24h")
print(f"Unique genes with good p-values {len(ubi_unique_genes_with_good_p_value_cov2)} on a total of {len(gene_list_6h_cov2) + len(gene_list_12h_cov2) + len(gene_list_24h_cov2)}")


In SARS-CoV-2 we have ubiquitinome regulated genes with p-value < 0.05 shared among the time points 6h,12h,24h = 166 on a total of 240 at 6h of 791 at 12h of 3181 at 24h
Unique genes with good p-values 1610 on a total of 4212


Case of SARS-CoV:

In [23]:
gene_list_6h_cov, gene_list_12h_cov, gene_list_24h_cov, ubi_shared_genes_with_good_p_value_cov, ubi_unique_genes_with_good_p_value_cov = omic_completeness_analyzer(ubiquitinome, "SARS_CoV")
print(f"In SARS-CoV we have ubiquitinome regulated genes with p-value < 0.05 shared among the time points 6h,12h,24h = {len(ubi_shared_genes_with_good_p_value_cov)} on a total of {len(gene_list_6h_cov)} at 6h of {len(gene_list_12h_cov)} at 12h of {len(gene_list_24h_cov)} at 24h")
print(f"Unique genes with good p-values {len(ubi_unique_genes_with_good_p_value_cov)} on a total of {len(gene_list_6h_cov) + len(gene_list_12h_cov) + len(gene_list_24h_cov)}")


In SARS-CoV we have ubiquitinome regulated genes with p-value < 0.05 shared among the time points 6h,12h,24h = 181 on a total of 290 at 6h of 669 at 12h of 2259 at 24h
Unique genes with good p-values 1315 on a total of 3218


# Phosphoproteome complete data analysis

In [24]:
phosphoproteome.columns;

In [25]:
num_nan_values_sars_cov_2 = phosphoproteome["fold_change_log2.SARS_CoV2@6h_vs_mock@6h"].isnull().sum() + phosphoproteome["fold_change_log2.SARS_CoV2@12h_vs_mock@12h"].isnull().sum() + phosphoproteome["fold_change_log2.SARS_CoV2@24h_vs_mock@24h"].isnull().sum()
num_nan_values_sars_cov = phosphoproteome["fold_change_log2.SARS_CoV@6h_vs_mock@6h"].isnull().sum() + phosphoproteome["fold_change_log2.SARS_CoV@12h_vs_mock@12h"].isnull().sum() + phosphoproteome["fold_change_log2.SARS_CoV@24h_vs_mock@24h"].isnull().sum()

print(f"number of nan values for SARS-CoV-2 {num_nan_values_sars_cov_2}")
print(f"number of nan values for SARS-CoV {num_nan_values_sars_cov}")

number of nan values for SARS-CoV-2 0
number of nan values for SARS-CoV 0


Case of SARS-CoV-2:

In [26]:
gene_list_6h_cov2, gene_list_12h_cov2, gene_list_24h_cov2, pho_shared_genes_with_good_p_value_cov2, pho_unique_genes_with_good_p_value_cov2 = omic_completeness_analyzer(phosphoproteome, "SARS_CoV2")
print(f"In SARS-CoV-2 we have phosphoproteome regulated genes with p-value < 0.05 shared among the time points 6h,12h,24h = {len(pho_shared_genes_with_good_p_value_cov2)} on a total of {len(gene_list_6h_cov2)} at 6h of {len(gene_list_12h_cov2)} at 12h of {len(gene_list_24h_cov2)} at 24h")
print(f"Unique genes with good p-values {len(pho_unique_genes_with_good_p_value_cov2)} on a total of {len(gene_list_6h_cov2) + len(gene_list_12h_cov2) + len(gene_list_24h_cov2)}")


In SARS-CoV-2 we have phosphoproteome regulated genes with p-value < 0.05 shared among the time points 6h,12h,24h = 1491 on a total of 1743 at 6h of 3284 at 12h of 5645 at 24h
Unique genes with good p-values 2604 on a total of 10672


Case of SARS-CoV

In [27]:
gene_list_6h_cov, gene_list_12h_cov, gene_list_24h_cov, pho_shared_genes_with_good_p_value_cov, pho_unique_genes_with_good_p_value_cov = omic_completeness_analyzer(phosphoproteome, "SARS_CoV")
print(f"In SARS-CoV we have phosphoproteome regulated genes with p-value < 0.05 shared among the time points 6h,12h,24h = {len(pho_shared_genes_with_good_p_value_cov)} on a total of {len(gene_list_6h_cov)} at 6h of {len(gene_list_12h_cov)} at 12h of {len(gene_list_24h_cov)} at 24h")
print(f"Unique genes with good p-values {len(pho_unique_genes_with_good_p_value_cov)} on a total of {len(gene_list_6h_cov) + len(gene_list_12h_cov) + len(gene_list_24h_cov)}")


In SARS-CoV we have phosphoproteome regulated genes with p-value < 0.05 shared among the time points 6h,12h,24h = 1074 on a total of 1316 at 6h of 2593 at 12h of 4726 at 24h
Unique genes with good p-values 2326 on a total of 8635


## Dataset creation SARS-CoV-2

effectome, proteome, phosphoproteome, ubiquitinome, transcriptome. 
Lets collect all the genes found in either one of the omics.

In [5]:
import numpy as np

def add_string_items_only(list_to_fill, list_to_check):
    for item in list_to_check:
        if type(item) == str:
            list_to_fill.append(item)

effectome_host_genes = []
add_string_items_only(effectome_host_genes, effectome[effectome["Viral protein"] == "SARS-CoV-2"]["Host protein"].unique()[1:])
        
proteome_host_genes = []
add_string_items_only(proteome_host_genes, proteome["Gene.names"].unique())
                     
phosphoproteome_host_genes = []
add_string_items_only(phosphoproteome_host_genes, phosphoproteome[phosphoproteome["is_viral"] == False]["gene_name"].unique())
                     
ubiquitinome_host_genes = []
add_string_items_only(ubiquitinome_host_genes, ubiquitinome[ubiquitinome["is_viral"] == False]["gene_name"].unique())
        
transcriptome_host_genes = []
add_string_items_only(transcriptome_host_genes, transcriptome["gene_name"].unique())

In [6]:
all_protein_coding_genes = np.unique(np.append(np.append(np.append(effectome_host_genes, proteome_host_genes), phosphoproteome_host_genes),ubiquitinome_host_genes))

In [7]:
for item in all_protein_coding_genes:
    if item == "RPS17":
        print("eccomi")

eccomi


Let's start creating the most complete dataset, 
- setting all the missing feature data to zero
- deleting all the proteins found in STRING PPI network but not in my most complete list of proteins taken from all the omics  

I start by downloading the database of the human STRING protein IDs with the corresponding gene names

In [8]:
from pybiomart import Server, Dataset

In [9]:
dataset = Dataset(name='hsapiens_gene_ensembl',
                  host='http://www.ensembl.org')

server = Server(host='http://www.ensembl.org')
dataset_server =  (server.marts['ENSEMBL_MART_ENSEMBL'].datasets['hsapiens_gene_ensembl'])

In [10]:
df_from_protein_id_to_gene_name = dataset_server.query(attributes=['ensembl_peptide_id', 'external_gene_name'])

In [11]:
df_from_protein_id_to_gene_name.head()

,Protein stable ID,Gene name
0,NaN,MT-TF
1,NaN,MT-RNR1
2,NaN,MT-TV
3,NaN,MT-RNR2
4,NaN,MT-TL1


In [12]:
#df_from_protein_id_to_gene_name = dataset.query(attributes=['ensembl_peptide_id', 'external_gene_name'])
#df_from_protein_id_to_gene_name.head()

- I need now to import the PPI network from STRING

In [12]:
df_string_ppi = pd.read_csv("9606.protein.physical.links.detailed.v11.5.txt", sep = " ")
df_string_ppi.head()

,protein1,protein2,experimental,database,textmining,combined_score
0,9606.ENSP00000000233,9606.ENSP00000264718,157,0,0,156
1,9606.ENSP00000000233,9606.ENSP00000346046,178,0,0,177
2,9606.ENSP00000000233,9606.ENSP00000347134,143,0,63,162
3,9606.ENSP00000000233,9606.ENSP00000317469,345,0,92,379
4,9606.ENSP00000000233,9606.ENSP00000302393,288,0,0,287


- remove all the lines with $ experimental < 10$

In [13]:
len(df_string_ppi["experimental"])

1991832

In [15]:
counter = 0
index_list = []
for index, row in df_string_ppi.iterrows():
    if row["experimental"] < 10:
        counter += 1
        index_list.append(index)
        if counter % 10000 == 0:    
            print(counter)
        
print(counter)        

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
236632


In [16]:
df_string_ppi.drop(index_list, axis = 0, inplace = True)

In [17]:
df_string_ppi.reset_index(drop = True, inplace = True)

In [18]:
df_string_ppi.head()

,protein1,protein2,experimental,database,textmining,combined_score
0,9606.ENSP00000000233,9606.ENSP00000264718,157,0,0,156
1,9606.ENSP00000000233,9606.ENSP00000346046,178,0,0,177
2,9606.ENSP00000000233,9606.ENSP00000347134,143,0,63,162
3,9606.ENSP00000000233,9606.ENSP00000317469,345,0,92,379
4,9606.ENSP00000000233,9606.ENSP00000302393,288,0,0,287


In [14]:
df_from_protein_id_to_gene_name[df_from_protein_id_to_gene_name["Protein stable ID"] == "ENSP00000346046"]["Gene name"].values[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [21]:
df_from_protein_id_to_gene_name[df_from_protein_id_to_gene_name["Gene name"] == "AAMP"]["Protein stable ID"]

147208    ENSP00000248450
147209    ENSP00000403343
147210                NaN
147211    ENSP00000416394
147212    ENSP00000396295
147213    ENSP00000393818
Name: Protein stable ID, dtype: object

- ensamble.org is not complete enough, proteins present in STRING PPI network and in my list of "all_protein_coding_genes" are not listed in ensamble
- I will create a dictionary protein IDs to Gene Names from uniprot database

In [15]:
import Bio.SwissProt as sp
import csv

In [16]:
datContent = [i.strip().split() for i in open("./HUMAN_9606_idmapping.dat").readlines()]

In [17]:
# filter gene names and STRING names
string_and_gene_names = []
for item in datContent:
    if (item[1] == "Gene_Name") or (item[1] == "STRING"):
        string_and_gene_names.append(item)

In [18]:
uniprot_names = []
for item in string_and_gene_names:
    uniprot_names.append(item[0])
    
used_names = set()    
unique_uniprot_names = [elem for elem in uniprot_names if elem not in used_names and (used_names.add(elem) or True)]    
 

In [19]:
string_names_list = []
gene_names_list = []
counter = 0
for elem in string_and_gene_names:
    if elem[1] == "STRING":
        for names in string_and_gene_names:
            if names[0] == elem[0] and names[1] == "Gene_Name":
                string_names_list.append(elem[2])
                gene_names_list.append(names[2])
                break
    counter += 1
    if counter % 10000 == 0:
        print(counter)
                

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000


In [20]:
string_to_gen_names_map = {"string_ids" : string_names_list, "gene_names" : gene_names_list}

In [21]:
df_string_to_gen_names_map = pd.DataFrame(string_to_gen_names_map)

In [22]:
df_string_to_gen_names_map.head()

,string_ids,gene_names
0,9606.ENSP00000361930,YWHAB
1,9606.ENSP00000264335,YWHAE
2,9606.ENSP00000248975,YWHAH
3,9606.ENSP00000306330,YWHAG
4,9606.ENSP00000340989,SFN


In [23]:
df_string_ppi.head()

,protein1,protein2,experimental,database,textmining,combined_score
0,9606.ENSP00000000233,9606.ENSP00000264718,157,0,0,156
1,9606.ENSP00000000233,9606.ENSP00000346046,178,0,0,177
2,9606.ENSP00000000233,9606.ENSP00000347134,143,0,63,162
3,9606.ENSP00000000233,9606.ENSP00000317469,345,0,92,379
4,9606.ENSP00000000233,9606.ENSP00000302393,288,0,0,287


In [24]:
df_string_to_gen_names_map[df_string_to_gen_names_map["string_ids"] == "9606.ENSP00000346046"]["gene_names"].values[0]

'RPS17'

- let's create the gene names list corresponding to columns "protein1" and "protein2" in df_string_ppi

In [48]:
gene_names_1 = []
gene_names_2 = []

for index, row in df_string_ppi.iterrows():
    names_1 = df_string_to_gen_names_map[df_string_to_gen_names_map["string_ids"] == row["protein1"]]["gene_names"]
    names_2 = df_string_to_gen_names_map[df_string_to_gen_names_map["string_ids"] == row["protein2"]]["gene_names"]
    if len(names_1) > 0:
        gene_names_1.append(names_1.values[0])
    else:
        gene_names_1.append('nan')
    if len(names_2) > 0:    
        gene_names_2.append(names_2.values[0])
    else:
        gene_names_2.append('nan')
    if index % 10000 == 0:
        print(index)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

In [58]:
counter = 0
for item in gene_names_2:
    if item == "nan":
        counter += 1
print(f"Percentage of not itentified gene names in STRING PPI dataset: {100*counter/len(gene_names_2)}%")

Percentage of not itentified gene names in STRING PPI dataset: 2.682543299908842%


In [66]:
df_string_ppi["gene_name_1"] = pd.Series(gene_names_1, index = df_string_ppi.index)

- I add the corresponding gene names to the protein interaction network 

In [67]:
df_string_ppi["gene_name_2"] = pd.Series(gene_names_2, index = df_string_ppi.index)
df_string_ppi.head()

,protein1,protein2,experimental,database,textmining,combined_score,gene_name_1,gene_name_2
0,9606.ENSP00000000233,9606.ENSP00000264718,157,0,0,156,ARF5,GPN1
1,9606.ENSP00000000233,9606.ENSP00000346046,178,0,0,177,ARF5,RPS17
2,9606.ENSP00000000233,9606.ENSP00000347134,143,0,63,162,ARF5,NET1
3,9606.ENSP00000000233,9606.ENSP00000317469,345,0,92,379,ARF5,BBS1
4,9606.ENSP00000000233,9606.ENSP00000302393,288,0,0,287,ARF5,LDHAL6B


In [25]:
# save the protein interaction network to a file 
#df_string_ppi.to_csv('df_string_ppi.zip', index=False, compression = dict(method='zip',archive_name='df_string_ppi.csv'))
df_string_ppi = pd.read_csv("df_string_ppi.zip")

In [26]:
transcriptome_host_genes = np.array(transcriptome_host_genes[:-2])

In [27]:
all_genes = np.unique(np.append(all_protein_coding_genes, all_protein_coding_genes))
len(all_genes)

9467

- check gene names in PPI network that are not present in my list of gene names

In [28]:
counter = 0
list_of_row_indices_to_drop = []
for index, row in df_string_ppi.iterrows():
    if (row["gene_name_1"] not in all_genes) or (row["gene_name_2"] not in all_genes):
        counter += 1
        list_of_row_indices_to_drop.append(index)
        if counter % 1000 == 0:    
            print(counter)
        
print(counter)  

/home/samuele/.pyenv/versions/3.6.10/envs/jupiter-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  after removing the cwd from sys.path.


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [29]:
print(f"Percentage of lines to drop from the PPI network: {100*len(list_of_row_indices_to_drop)/len(df_string_ppi)}%")

Percentage of lines to drop from the PPI network: 48.440861440291705%


- I drop the links with proteins not present at all in my dataset 

In [30]:
df_string_ppi.drop(list_of_row_indices_to_drop, axis = 0, inplace = True)

In [31]:
df_string_ppi.reset_index(drop = True, inplace = True)

In [32]:
df_string_ppi.head()

,protein1,protein2,experimental,database,textmining,combined_score,gene_name_1,gene_name_2
0,9606.ENSP00000000233,9606.ENSP00000264718,157,0,0,156,ARF5,GPN1
1,9606.ENSP00000000233,9606.ENSP00000346046,178,0,0,177,ARF5,RPS17
2,9606.ENSP00000000233,9606.ENSP00000347134,143,0,63,162,ARF5,NET1
3,9606.ENSP00000000233,9606.ENSP00000354876,153,0,0,152,ARF5,MT-CO2
4,9606.ENSP00000000233,9606.ENSP00000381607,169,0,0,169,ARF5,GSTP1


In [33]:
print(f"Total number of links {len(df_string_ppi)}")
node_genes = np.unique(np.append(df_string_ppi["gene_name_2"].unique(), df_string_ppi["gene_name_1"].unique()))
print(f"Total number of nodes {len(node_genes)}")
print(f"Total number of starting proteins for which I have at least some feature data {len(all_genes)}")

Total number of links 904966
Total number of nodes 8957
Total number of starting proteins for which I have at least some feature data 9467


- let's evaluate the percentage of not identified genes using "ensamble.org"

In [96]:
df_string_ppi_original = pd.read_csv("df_string_ppi.zip")

In [97]:
len(df_string_ppi_original)

1755200

In [98]:
df_string_ppi_original.head()

,protein1,protein2,experimental,database,textmining,combined_score,gene_name_1,gene_name_2
0,9606.ENSP00000000233,9606.ENSP00000264718,157,0,0,156,ARF5,GPN1
1,9606.ENSP00000000233,9606.ENSP00000346046,178,0,0,177,ARF5,RPS17
2,9606.ENSP00000000233,9606.ENSP00000347134,143,0,63,162,ARF5,NET1
3,9606.ENSP00000000233,9606.ENSP00000317469,345,0,92,379,ARF5,BBS1
4,9606.ENSP00000000233,9606.ENSP00000302393,288,0,0,287,ARF5,LDHAL6B


In [ ]:
gene_names_from_ensamble_1 = []
gene_names_from_ensamble_2 = []

for index, row in df_string_ppi.iterrows():
    names_1 = df_from_protein_id_to_gene_name[df_from_protein_id_to_gene_name["Protein stable ID"] == row["protein1"]]["Gene name"]
    names_2 = df_from_protein_id_to_gene_name[df_from_protein_id_to_gene_name["Protein stable ID"] == row["protein2"]]["Gene name"]
    if len(names_1) > 0:
        gene_names_from_ensamble_1.append(names_1.values[0])
    else:
        gene_names_from_ensamble_1.append('nan')
    if len(names_2) > 0:    
        gene_names_from_ensamble_2.append(names_2.values[0])
    else:
        gene_names_from_ensamble_2.append('nan')
    if index % 10000 == 0:
        print(index)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
